## Data Cleaning Accidents BCN:
#### In this Jupyter Notebook you will find the process of data cleaning of a dataset from Ajuntament de Barcelona Open Data on vehicles involved in accident.

First we import the libraries we are going to use.

In [1]:
import numpy as np
import pandas as pd
import re

Import data and assign to variable 'acc' which stands for accident.

In [2]:
acc = pd.read_csv('../Data/Raw_data/vehicles_accident.csv', encoding='utf-8', sep = ';',dtype = 'object')

I examine the data that I have imported.

In [3]:
acc.sample(5)

,Codi_expedient,Codi_districte,Nom_districte,Codi_barri,Nom_barri,Codi_carrer,Nom_carrer,Num_postal,Descripcio_dia_setmana,Dia_setmana,...,Descripcio_tipus_vehicle,Descripcio_model,Descripcio_marca,Descripcio_color,Descripcio_carnet,Antiguitat_carnet,Coordenada_UTM_X,Coordenada_UTM_Y,Longitud,Latitud
81852,2014S007872,7,Horta-Guinardó,41,la Vall d'Hebron,352507,Vall d'Hebron,0202 0202,Dimecres,Dc,...,Turismo,CORDOBA,SEAT,Verd,B,22,"4587166,38","428713,22",NaN,NaN
64570,2015S009704,4,Les Corts,20,la Maternitat i Sant Ramon,100708,Arístides Maillol,0006 0018,Dilluns,Dl,...,Bicicleta,Desconegut,BH,Vermell,Desconegut,Desconegut,"4581415,28","426532,37",NaN,NaN
18665,2018S007810,2,Eixample,5,el Fort Pienc,169409,Corts Catalanes ...,790,Diumenge,Dg,...,Furgoneta,JUMPER,CITROEN,Blanc,B,41,431743.09,4583649.26,2.182.325,41.399.642
37989,2017S010311,2,Eixample,10,Sant Antoni,361406,Viladomat,0081 0083,Dimecres,Dc,...,Taxi,OCTAVIA,SKODA,Negre/Groc,B,28,"429717,35","4581348,59",2.158.357,41.378.741
60858,2015S009437,10,Sant Martí,68,el Poblenou,30500,Badajoz,0006 0006,Divendres,Dv,...,Motocicleta,YP 125,YAMAHA,Desconegut,B,19,"4583104,53","433261,88",NaN,NaN


As I want to study which common characteristics the vehicles involved in an accident have, there are some columns that I won't use, so I will drop them.

In [4]:
acc.columns #Checking the names of the columns.

Index(['Codi_expedient', 'Codi_districte', 'Nom_districte', 'Codi_barri',
       'Nom_barri', 'Codi_carrer', 'Nom_carrer', 'Num_postal ',
       'Descripcio_dia_setmana', 'Dia_setmana', 'Descripcio_tipus_dia', 'Any',
       'Mes_any', 'Nom_mes', 'Dia_mes', 'Hora_dia', 'Descripcio_torn',
       'Descripcio_causa_vianant', 'Descripcio_tipus_vehicle',
       'Descripcio_model', 'Descripcio_marca', 'Descripcio_color',
       'Descripcio_carnet', 'Antiguitat_carnet', 'Coordenada_UTM_X',
       'Coordenada_UTM_Y', 'Longitud', 'Latitud'],
      dtype='object')

In [5]:
#Dropping columns that I will not be using.
acc.drop(columns = ['Codi_districte', 'Nom_districte', 'Codi_barri', \
                    'Nom_barri', 'Codi_carrer', 'Nom_carrer', 'Num_postal ', \
                    'Descripcio_dia_setmana', 'Dia_setmana', 'Descripcio_tipus_dia', 'Any', \
                    'Mes_any', 'Nom_mes', 'Dia_mes', 'Hora_dia', 'Descripcio_torn',\
                    'Descripcio_causa_vianant','Descripcio_carnet','Coordenada_UTM_X',\
                    'Coordenada_UTM_Y', 'Longitud', 'Latitud','Descripcio_model' ], inplace = True)

I will now change the name of the columns so that they are more self-explanatory.

In [6]:
acc.rename(columns = {'Codi_expedient': 'Accident_code', 'Descripcio_tipus_vehicle':'Vehicle_type', 
                      'Descripcio_marca':'Brand','Descripcio_color':'Color', 'Antiguitat_carnet':'Years_license'},
          inplace = True)

Checking datatypes. I will change the type of Years_license at the end, since it has some values as Desconegut (Unknown) that don't let me change directly the data type.

In [7]:
acc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 96125 entries, 0 to 96124
Data columns (total 5 columns):
Accident_code    96124 non-null object
Vehicle_type     96124 non-null object
Brand            96124 non-null object
Color            96124 non-null object
Years_license    96124 non-null object
dtypes: object(5)
memory usage: 3.7+ MB


##### Vehicle_type column:

Moreover, I will change the values of the columns Vehicle_type and Color to make it more understandable. First, I will explore which are the values of Vehicle_Type.

In [8]:
acc['Vehicle_type'].value_counts() #Checking the values of Vehicle_type.

Motocicleta                            30293
Turismo                                25908
Turisme                                13616
Furgoneta                               6128
Ciclomotor                              5332
Taxi                                    5092
Bicicleta                               3849
Autobús                                 1995
Camión <= 3,5 Tm                         952
AutobÃºs                                 476
Camión > 3,5 Tm                          423
CamiÃ_ rÃ_gid <= 3,5 tones               280
Camió rígid <= 3,5 tones                 208
CamiÃ_ rÃ_gid > 3,5 tones                169
Autocar                                  161
Tractocamión                             150
Todo terreno                             148
Veh. mobilitat personal amb motor        128
Camió rígid > 3,5 tones                   98
Otros vehíc. a motor                      86
Tot terreny                               80
Altres vehicles amb motor                 62
AutobÃºs a

Since I am studying the common elements between accidents, I will get rid of all the values except for cars and bikes. First, I will unify the names to Cars and bikes and then I will get rid of the other types of vehicles.

In [9]:
acc['Vehicle_type'].replace({'Turisme':'Car', 'Turismo':'Car', 'Todo terreno':'Car', \
                             'Tot terreny':'Car','Motocicleta':'Bike','Ciclomotor':'Bike'}, inplace = True)

Now I am getting rid of the other types of vehicles different than Car and Bike.

In [10]:
#Updating variable acc since it is the only information that we will be using.
acc = acc.query('Vehicle_type == "Car" | Vehicle_type == "Bike"')

##### Brand Column

Let's move to the column Brand. We can see that there are many brands with points and spaces, so I will get rid of them.

In [11]:
list(acc['Brand'].unique()) 
#Seeing the unique values for Brands, if I used value_counts, it didn't print all the values.

['SEAT',
 'PEUGEOT',
 'RENAULT',
 'Desconegut',
 'FORD',
 'HONDA',
 'SYM',
 'BMW',
 'A.U.D.I.',
 'OPEL',
 'PIAGGIO',
 'KIA',
 'YAMAHA',
 'VOLKSWAGEN',
 'SKODA',
 'APRILIA',
 'HARLEY DAVIDSON',
 'MERCEDES',
 'EMCO',
 'KAWASAKI',
 'LANCIA',
 'AUDI',
 'KYMCO',
 'FIAT',
 'TOYOTA',
 'MAZDA',
 'KEEWAY',
 'CITROEN',
 'DAEWOO',
 'KTM',
 'A U D I',
 'DAELIM',
 'S.Y.M',
 'DACIA',
 'DODGE',
 'NISSAN',
 'HYUNDAI',
 'DERBI',
 'SUZUKI',
 'LAND ROVER',
 'GILERA',
 'MONTESA',
 'MINI',
 'HYOSUNG',
 'ALFA ROMEO',
 'GOVECS',
 'SCUTUM',
 'TRIUMPH',
 'SMART',
 'YIYING',
 'VOLVO',
 'JEEP',
 'TATA',
 'LEXUS',
 'HUSQVARNA',
 'PORSCHE',
 'MITSUBISHI',
 'CHEVROLET',
 'BENELLI',
 'MASH',
 'SUMCO',
 'BUELL',
 'B.M.W.',
 'CHRYSLER',
 'VESPA',
 'ES DESCONEIX',
 'Brixton',
 'JAGUAR',
 'B M W',
 'RIEJU',
 'DUCATI',
 'SSANGYONG',
 'BETA',
 'SAAB',
 'SUBARU',
 'HANWAY',
 'INFINITI',
 'COOLTRA',
 'IRISBUS',
 'GAS-GAS',
 'MERCEDES-BENZ',
 'TISONG',
 'MBK',
 'MV AUGUSTA',
 'CPI',
 'GENERIC',
 'GUZZI',
 'WOTTAN',
 'SCOMADI

In [12]:
acc['Brand'] = acc['Brand'].apply(lambda x: x.replace('.','')) #Getting rid of points.

In [13]:
acc['Brand'] = acc['Brand'].apply(lambda x: x.replace(' ','')) #Getting rid of additional spaces.

There are some brands that have more information in the brand name, hence, I will only keep the brand name.

In [14]:
acc['Brand'] = acc['Brand'].apply(lambda x: re.sub('MERCED.*','MERCEDES',x)) #Doing the above explained to MERCEDES.

In [15]:
acc['Brand'] = acc['Brand'].apply(lambda x: re.sub('JAGUAR.*','JAGUAR',x)) #Doing the above explained to JAGUAR.

In [16]:
acc['Brand'].replace({'ESDESCONEIX':'Unknown','Desconegut':'Unknown'}, inplace = True) #Uniforming unknown values.

##### Column Color:

In [17]:
acc['Color'].unique()

array(['Blanc', 'Blau', 'Negre', 'Desconegut', 'Vermell', 'Gris', 'Verd',
       'MarrÃ_', 'Platejat', 'Taronja', 'granate', 'Altres', 'Violeta',
       'Groc', 'Negre/Groc', 'Beige', 'Daurat', 'Rosa', 'Marró'],
      dtype=object)

There are some values that have special characters and I have decided to translate it into English.

In [18]:
acc['Color'].replace({'Blanc':'White', 'Blau':'Blue', 'Negre':'Black', 'Desconegut':'Unknown',
                      'Vermell':'Red', 'Gris':'Grey', 'Verd':'Green','MarrÃ_':'Brown',
                      'Platejat':'Silver', 'Taronja':'Orange', 'granate':'Red', 'Altres':'Unknown',
                      'Violeta':'Purple','Groc':'Yellow', 'Negre/Groc':'Black-Yellow',
                      'Beige':'White', 'Daurat':'Golden', 'Rosa':'Purple', 'Marró':'Brown'}, inplace = True)

##### Column Years_license:

In [19]:
acc['Years_license'].unique()

array(['5', '1', '2', 'Desconegut', '19', '12', '10', '23', '18', '22',
       '4', '11', '13', '17', '21', '29', '28', '14', '8', '15', '32',
       '34', '7', '46', '38', '6', '3', '9', '20', '50', '0', '25', '16',
       '27', '33', '39', '40', '24', '35', '41', '36', '52', '31', '30',
       '26', '51', '37', '58', '48', '45', '42', '44', '43', '54', '47',
       '60', '56', '53', '55', '49', '57', '61', '64', '63', '65', '68',
       '59', '66', '71', '67', '62', '-46', '69', '-189', '73', '-5',
       '-47', '-2', '1818', '-484', '-972', '-70', '-947', '-41', '-48',
       '-45', '-28', '1018', '-969'], dtype=object)

Let's get rid of the 'Desconegut' values, I will first assign them to a -999 values so that I continue having easy track of them.

In [20]:
acc['Years_license'].replace({'Desconegut':'-999'}, inplace = True)

Now, I will change the datatype into integer.

In [35]:
acc['Years_license'] = acc['Years_license'].astype('int')

In [32]:
acc['Years_license']

0           5
1           1
6           2
8        -999
9          19
         ... 
96119      31
96120      13
96122       1
96123      36
96124      39
Name: Years_license, Length: 75377, dtype: object

There are some strange values (negative & >1000 years). Let's examine them:

In [22]:
acc.query('Years_license < "0"')

,Accident_code,Vehicle_type,Brand,Color,Years_license
8,2018S006793,Car,Unknown,Unknown,-999
14,2018S009491,Bike,SYM,White,-999
15,2018S009491,Car,BMW,Grey,-999
58,2018S003738,Bike,HONDA,White,-999
60,2018S003738,Car,Unknown,Unknown,-999
...,...,...,...,...,...
96093,2014S001775,Bike,Unknown,Unknown,-999
96099,2014S008681,Bike,HONDA,Black,-999
96104,2014S008826,Car,HYUNDAI,Green,-999
96109,2014S000418,Car,MAZDA,Grey,-999


In [23]:
acc.query('Years_license == "1818"')

,Accident_code,Vehicle_type,Brand,Color,Years_license
45980,2016S008427,Bike,KYMCO,Unknown,1818


They are only 21 rows, so I will delete them because it seems a typo, but I don't have a way to assess the value.

In [30]:
len(acc.query('Brand == "Unknown" & Color == "Unknown" & Years_license < "0"'))*100/len(acc)

0.7840587977765101